In [1]:
import os
from multiple_layout_generator import nmos_cm
from pex.extract import *

### Create Current mirror Layouts

In [2]:
nmos_cm(12, 12, 14e-9, source_first=True)

In [3]:
# Specify the directory path
path = 'outputs/'
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for layouts in files:
    file_name = path+files[0].split('.')[0]
    print(file_name)
    pex(file_name)